<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/ampds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
import io
from google.colab import files

In [0]:
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
uploaded = files.upload()
'''Load data'''
WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))['P']
uploaded = files.upload()
CWE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CWE.csv']))['P']
'''
CDE_data = pd.read_csv(base_dir+'Electricity_CDE.csv')['P']
CWE_data = pd.read_csv(base_dir+'Electricity_CWE.csv')['P']
DWE_data = pd.read_csv(base_dir+'Electricity_DWE.csv')['P']
FRE_data = pd.read_csv(base_dir+'Electricity_FRE.csv')['P']
HPE_data = pd.read_csv(base_dir+'Electricity_HPE.csv')['P']
HTE_data = pd.read_csv(base_dir+'Electricity_HTE.csv')['P']
'''

In [0]:
'''Hyper parameter'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
window_size=60
batch_size=512
unit_num=128

In [0]:
'''Using sliding window to build training examples'''
dataX = create_dataset(WHE_data.as_matrix(), window_size)

#get y for the single load
#cde_Y = CDE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
#cdeY=cde_Y[cde_Y>0].reshape(-1,1)
#cdeX=dataX[[np.where(cde_Y>0)[0]],:][0]

cwe_Y = CWE_data[window_size-1:].values.reshape([CWE_data.shape[0]-window_size+1,1])
cweY=cwe_Y[cwe_Y>0].reshape(-1,1)
cweX=dataX[[np.where(cwe_Y>0)[0]],:][0]

'''
cwe_Y = CWE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
dwe_Y = DWE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
fre_Y = FRE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
hpe_Y = HPE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
hte_Y = HTE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
dataY = np.concatenate((cde_Y,cwe_Y,dwe_Y,fre_Y,hpe_Y,hte_Y),axis=1)
'''

In [0]:

X_train, X_test, y_train, y_test = train_test_split(cweX, cweY, test_size=0.01, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
'''RNN Model Definition'''
tf.reset_default_graph()
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1])
tf_y = tf.placeholder(tf.int32, [None, 1])
'''
def add_layer(name1,inputs,in_size,out_size,activation_function=None):
    Weights = tf.get_variable(name1,[in_size, out_size], initializer = tf.ini)
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
'''
#Transform from 3d to 2d
#X = tf.reshape(tf_x, [-1, 1])
#X = tf.reshape(tf_x, [-1, window_size])


'''
add_layer('l1',X,window_size, 128, activation_function=tf.nn.sigmoid)
l2 = add_layer('l2',l1, 128,32 , activation_function=tf.nn.sigmoid)
l3 = add_layer('l3',l2, 32,8 , activation_function=tf.nn.sigmoid)
pred = add_layer('l4',l3, 8,1 , activation_function=None)
'''
#X_in = tf.reshape(tf_x, [-1, 1])
lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num)
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],64,activation=tf.nn.leaky_relu)
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu)
l3 = tf.layers.dense(l2,16,activation=tf.nn.leaky_relu)
l4 = tf.layers.dense(l3,8,activation=tf.nn.leaky_relu)
l5 = tf.layers.dense(l4,4,activation=tf.nn.leaky_relu)
#l6 = tf.layers.dense(l5,32,activation=tf.nn.leaky_relu)
#l7 = tf.layers.dense(l2,8,activation=tf.nn.leaky_relu)
pred = tf.layers.dense(l5,1,activation=tf.nn.relu)
#pred = add_layer('dense_1',outputs[:, -1, :], unit_num, 1, activation_function=tf.nn.leaky_relu)
'''
#dense_2 = add_layer('dense_2',dense_1, 32, 16, activation_function=tf.nn.leaky_relu)
#dense_3 = add_layer('dense_3',dense_2, 16, 4, activation_function=tf.nn.leaky_relu)
#dense_4 = add_layer('dense_4',dense_1, 1024, 512, activation_function=tf.nn.relu)
#dense_5 = add_layer('dense_5',dense_4, 512, 256, activation_function=tf.nn.relu)
#dense_6 = add_layer('dense_6',dense_5, 256, 128, activation_function=tf.nn.relu)
#dense_7 = add_layer('dense_7',dense_1, 128, 64, activation_function=tf.nn.relu)
#dense_8 = add_layer('dense_8',dense_7, 64, 16, activation_function=tf.nn.relu)
#dense_9 = add_layer('dense_9',dense_1, 8, 4, activation_function=tf.nn.relu)
#pred = add_layer('output', dense_3,4, 1, activation_function=None)
'''
with tf.name_scope('loss'):
    loss =  tf.reduce_mean(tf.losses.mean_squared_error(tf_y, pred)) 
    #sigmoid_cross_entropy_with_logits(labels=tf_y, logits=pred)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)
'''
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs) 
'''
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
sess = tf.Session()
sess.run(init_op)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logss/", sess.graph) # tensorflow >=0.12

In [0]:
batch_num = len(X_train)//batch_size
for j in range(0,6000):
    for i in range(0,batch_num+1):
        if(i!=batch_num):
            batch_X = X_train[i*batch_size:(i+1)*batch_size,].reshape([batch_size,window_size,1])
            batch_Y = y_train[i*batch_size:(i+1)*batch_size,]
        else: 
            batch_X = X_train[-batch_size:].reshape([batch_size,window_size,1])
            batch_Y = y_train[-batch_size:,]
        #tmp = cf.get_change_points_binary_vector(batch_X)
        sess.run(train_op,{tf_x:batch_X,tf_y:batch_Y})
        cost_ = sess.run(loss, {tf_x: batch_X, tf_y:batch_Y})
        if i%10 == 0:
            rs = sess.run(merged,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            writer.add_summary(rs, i+j*batch_num)
            print('###############epoch: '+str(j)+' iter: '+str(i)+' ,train loss: %.4f' % cost_+'#############')
            #vars = tf.trainable_variables()
            #print(vars) #some infos about variables...
            #vars_vals = sess.run(vars)
            #for var, val in zip(vars, vars_vals):
            #    print("var: {}, value: {}".format(var.name, val))
            #print(pred.eval(session=sess, feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]),tf_y: y_test}))
loss = sess.run(loss, feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
print('test accuracy: %.2f'% loss)
print(y_test)
print(pre)
sess.close()
#tensorboard --logdir==mylogs:D:\tech\Workspaces\eclipse-workspace\NILM\logss


 > Downloading "/Electricity_CWE.csv" to "/content/Dropbox-Uploader/Electricity_CWE.csv"... DONE
